In [1]:
from requests_html import HTMLSession, HTML
from selenium import webdriver
import time
import re

In [2]:
#Kiểm tra xem dữ liệu có hợp pháp không
import urllib.robotparser
rp = urllib.robotparser.RobotFileParser()
rp.set_url('https://www.worldweatheronline.com/robots.txt')
rp.read()
rp.can_fetch('*', 'https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx')

True

In [3]:
session = HTMLSession()

In [4]:
url = "https://www.worldweatheronline.com/ho-chi-minh-city-weather-history/vn.aspx"

In [5]:
#Tên file chứa data
filename='weather_data'

In [6]:
#tập hợp các loại thời tiết (để phân loại và chuyển về 1 số loại nhất định)
list_w = set()

In [7]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

In [13]:
def parse_data(url, filename, year):
    name = f"{filename}_{year}.csv"
    # Specify the path to chromedriver.exe
    chrome_path = './chromedriver.exe'

    # Create a Service object with the path to chromedriver.exe
    service = Service(chrome_path)

    # Use the Service object when creating the Chrome WebDriver
    chrome_options = Options()
    chrome_options.add_argument('--headless')  # Add this line for headless mode
    browser = webdriver.Chrome(service=service, options=chrome_options)
    browser.get(url)

    for y in year:
        if y % 4 == 0 or y % 400 == 0:
            ferb_day = 30
        else:
            ferb_day = 29

        # danh sách ngày trong tháng (để tìm kiếm ngày trong web)
        # thêm 0 để chỉ số bằng số tháng
        day = [
            [0],
            range(1, 32),
            range(1, ferb_day),
            range(1, 32),
            range(1, 31),
            range(1, 32),
            range(1, 31),
            range(1, 32),
            range(1, 32),
            range(1, 31),
            range(1, 32),
            range(1, 31),
            range(1, 32)
        ]

        for m in range(12):
            m += 1
            if m == 12 and y == 2019:
                break  # Chỉ lấy đến tháng 11 năm 2019
            for d in day[m]:
                if d < 10:
                    d = f"0{d}"
                if m < 10:
                    m_str = f"0{m}"
                    text = f"{m_str}{d}{y}"
                else:
                    text = f"{m}{d}{y}"

                # Xác định vị trí của textbox và điền giá trị vào
                textbox = browser.find_element("xpath", "//input[@id='ctl00_MainContentHolder_txtPastDate']")
                textbox.click()
                print(text)
                textbox.send_keys(text)

                # Bấm nút để lấy dữ liệu
                button = browser.find_element("xpath", "//input[@id='ctl00_MainContentHolder_butShowPastWeather']")
                button.click()
                time.sleep(1)  # Nghỉ 1 giây để trang có thời gian load

                r = HTML(html=browser.page_source)

                # kiểm tra xem page đã load đúng chưa
                # nếu sai thì tăng thời gian sleep
                title = r.find(".block_title", first=True)
                if title is None:
                    print("Error: Block title not found.")
                    continue

                print(title.text)

                # lấy dữ liệu
                row = r.find(".days-comment", first=True)
                if row is None:
                    print("Error: Data row not found a.")
                    continue

                times = re.findall("\d{2}:\d{2}", row.text)

                row = r.find(".days-temp", first=True)
                if row is None:
                    print("Error: Temperature row not found b.")
                    continue

                temp = list(map(float, re.findall("(\d{1,2})", row.text)))

                row = r.find(".days-table-forecast-p", first=True)
                feels = list(map(float, re.findall("(\d{1,2})", row.text)))

                row = r.find(".days-wind-green", first=True).text
                wind = list(map(float, re.findall("(\d{1,2}) km/h", row)))
                direction = re.findall(r"\n([A-Z]{1,3})", row)

                row = r.find(".tb_gust", first=True).text
                gust = list(map(float, re.findall("\d{1,2}", row)))

                row = r.find(".tb_cloud", first=True).text
                cloud = list(map(float, re.findall("\d{1,2}", row)))

                row = r.find(".tb_humidity", first=True).text
                humidity = list(map(float, re.findall("\d{1,2}", row)))

                row = r.find(".tb_precip", first=True).text
                precip = list(map(float, re.findall("(\d{1}.\d{1,3})", row)))

                row = r.find(".tb_pressure", first=True).text
                pressure = list(map(float, re.findall("\d{1,4}", row)))

                weather = []
                row = r.find(".tb_weather", first=True)
                att = row.find(".tb_item_img")
                for a in att:
                    weather += [a.attrs['alt']]
                    # list_w.add(a.attrs['alt'])

                # ghi dữ liệu ra file csv
                with open(name, 'w') as f:
                    f.write(
                        'time\tmonth\ttemperature\tfeelslike\twind\tdirection\tgust\tcloud\thumidity\tprecipitation\tpressure\tweather\n')

                with open(name, "a") as f:
                    for i in range(8):
                        f.write(
                            f"{times[i]}\t{m}\t{temp[i]}\t{feels[i]}\t{wind[i]}\t{direction[i]}\t{gust[i]}\t{cloud[i]}\t{humidity[i]}\t{precip[i]}\t{pressure[i]}\t{weather[i]}\n")

Lưu dữ liệu ra 3 file riêng để nếu năm nào đó lấy bị lỗi chỉ cần crawl lại năm đó. Thỉnh thoảng driver chrome sẽ bị lỗi và trang không load được, sẽ có lỗi chính là các dòng ngày tháng in ra bị lặp lại

Ví dụ


12232017

Sat 23rd Dec, 2017

12242017

Sat 23rd Dec, 2017

12242017

Sat 23rd Dec, 2017


Nếu trong quá trình crawl gặp lỗi này thì chạy lại code và hạn chế tương tác với cửa số chrome bật lên, đồng thời, không được bấm nút minimize hoặc bất cư thao tác gì minimize cửa sổ đó (mở cửa sổ chrome khác để đè lên thì được)

In [14]:
parse_data(url,filename,[2023])

01012023
Ho Chi Minh City Historical Weather on 01st January over the years


AttributeError: 'NoneType' object has no attribute 'text'

In [15]:
parse_data(url,filename,[2018])

01012018
Mon 01st Jan, 2018
01022018
Tue 02nd Jan, 2018
01032018
Wed 03rd Jan, 2018
01042018
Thu 04th Jan, 2018
01052018
Fri 05th Jan, 2018
01062018
Sat 06th Jan, 2018
01072018
Sun 07th Jan, 2018
01082018
Mon 08th Jan, 2018
01092018
Tue 09th Jan, 2018
01102018
Wed 10th Jan, 2018
01112018
Thu 11th Jan, 2018
01122018
Fri 12th Jan, 2018
01132018
Sat 13th Jan, 2018
01142018
Sun 14th Jan, 2018
01152018
Mon 15th Jan, 2018
01162018
Tue 16th Jan, 2018
01172018
Wed 17th Jan, 2018
01182018
Thu 18th Jan, 2018
01192018
Fri 19th Jan, 2018
01202018
Sat 20th Jan, 2018
01212018
Sun 21st Jan, 2018
01222018
Mon 22nd Jan, 2018
01232018
Tue 23rd Jan, 2018
01242018
Wed 24th Jan, 2018
01252018
Thu 25th Jan, 2018
01262018
Fri 26th Jan, 2018
01272018
Sat 27th Jan, 2018
01282018
Sun 28th Jan, 2018
01292018
Mon 29th Jan, 2018
01302018
Tue 30th Jan, 2018
01312018
Wed 31st Jan, 2018
02012018
Thu 01st Feb, 2018
02022018
Fri 02nd Feb, 2018
02032018
Sat 03rd Feb, 2018
02042018
Sun 04th Feb, 2018
02052018
Mon 05th Fe

10212018
Sun 21st Oct, 2018
10222018
Mon 22nd Oct, 2018
10232018
Tue 23rd Oct, 2018
10242018
Wed 24th Oct, 2018
10252018
Thu 25th Oct, 2018
10262018
Fri 26th Oct, 2018
10272018
Sat 27th Oct, 2018
10282018
Sun 28th Oct, 2018
10292018
Mon 29th Oct, 2018
10302018
Tue 30th Oct, 2018
10312018
Wed 31st Oct, 2018
11012018
Thu 01st Nov, 2018
11022018
Fri 02nd Nov, 2018
11032018
Sat 03rd Nov, 2018
11042018
Sun 04th Nov, 2018
11052018
Mon 05th Nov, 2018
11062018
Tue 06th Nov, 2018
11072018
Wed 07th Nov, 2018
11082018
Thu 08th Nov, 2018
11092018
Fri 09th Nov, 2018
11102018
Sat 10th Nov, 2018
11112018
Sun 11th Nov, 2018
11122018
Mon 12th Nov, 2018
11132018
Tue 13th Nov, 2018
11142018
Wed 14th Nov, 2018
11152018
Thu 15th Nov, 2018
11162018
Fri 16th Nov, 2018
11172018
Sat 17th Nov, 2018
11182018
Sun 18th Nov, 2018
11192018
Mon 19th Nov, 2018
11202018
Tue 20th Nov, 2018
11212018
Wed 21st Nov, 2018
11222018
Thu 22nd Nov, 2018
11232018
Fri 23rd Nov, 2018
11242018
Sat 24th Nov, 2018
11252018
Sun 25th No

In [16]:
parse_data(url,filename,[2019])#nam 2019 chỉ lấy đến hết tháng 11

01012019
Tue 01st Jan, 2019
01022019
Wed 02nd Jan, 2019
01032019
Thu 03rd Jan, 2019
01042019
Fri 04th Jan, 2019
01052019
Sat 05th Jan, 2019
01062019
Sun 06th Jan, 2019
01072019
Mon 07th Jan, 2019
01082019
Tue 08th Jan, 2019
01092019
Wed 09th Jan, 2019
01102019
Thu 10th Jan, 2019
01112019
Fri 11th Jan, 2019
01122019
Sat 12th Jan, 2019
01132019
Sun 13th Jan, 2019
01142019
Mon 14th Jan, 2019
01152019
Tue 15th Jan, 2019
01162019
Wed 16th Jan, 2019
01172019
Thu 17th Jan, 2019
01182019
Fri 18th Jan, 2019
01192019
Sat 19th Jan, 2019
01202019
Sun 20th Jan, 2019
01212019
Mon 21st Jan, 2019
01222019
Tue 22nd Jan, 2019
01232019
Wed 23rd Jan, 2019
01242019
Thu 24th Jan, 2019
01252019
Fri 25th Jan, 2019
01262019
Sat 26th Jan, 2019
01272019
Sun 27th Jan, 2019
01282019
Mon 28th Jan, 2019
01292019
Tue 29th Jan, 2019
01302019
Wed 30th Jan, 2019
01312019
Thu 31st Jan, 2019
02012019
Fri 01st Feb, 2019
02022019
Sat 02nd Feb, 2019
02032019
Sun 03rd Feb, 2019
02042019
Mon 04th Feb, 2019
02052019
Tue 05th Fe

10212019
Mon 21st Oct, 2019
10222019
Tue 22nd Oct, 2019
10232019
Wed 23rd Oct, 2019
10242019
Thu 24th Oct, 2019
10252019
Fri 25th Oct, 2019
10262019
Sat 26th Oct, 2019
10272019
Sun 27th Oct, 2019
10282019
Mon 28th Oct, 2019
10292019
Tue 29th Oct, 2019
10302019
Wed 30th Oct, 2019
10312019
Thu 31st Oct, 2019
11012019
Fri 01st Nov, 2019
11022019
Sat 02nd Nov, 2019
11032019
Sun 03rd Nov, 2019
11042019
Mon 04th Nov, 2019
11052019
Tue 05th Nov, 2019
11062019
Wed 06th Nov, 2019
11072019
Thu 07th Nov, 2019
11082019
Fri 08th Nov, 2019
11092019
Sat 09th Nov, 2019
11102019
Sun 10th Nov, 2019
11112019
Mon 11th Nov, 2019
11122019
Tue 12th Nov, 2019
11132019
Wed 13th Nov, 2019
11142019
Thu 14th Nov, 2019
11152019
Fri 15th Nov, 2019
11162019
Sat 16th Nov, 2019
11172019
Sun 17th Nov, 2019
11182019
Mon 18th Nov, 2019
11192019
Tue 19th Nov, 2019
11202019
Wed 20th Nov, 2019
11212019
Thu 21st Nov, 2019
11222019
Fri 22nd Nov, 2019
11232019
Sat 23rd Nov, 2019
11242019
Sun 24th Nov, 2019
11252019
Mon 25th No

In [ ]:
name = f"{filename}_{year[0]}"
name